In [146]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import VarianceThreshold
import seaborn as sb
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

In [147]:
# Импортирование CSV
df = pd.read_csv('student-mat.csv', sep = ',') # DataFrame

# Определяем наш целевой столбец и порог для успешной сдачи
target_column_name = 'G3'
passing_grade_threshold = 5  # Пример порога

df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


О наборе данных
=========
# Контекст:
Данные были получены в ходе опроса учащихся курсов математики и португальского языка в средней школе. Он содержит много интересной социальной, гендерной и учебной информации о студентах.

# Содержание:
Атрибуты для наборов данных Student-mat.csv (курс математики) и Student-por.csv (курс португальского языка):
```
school - школа ученика (двоичный код: 'GP' - Габриэль Перейра или 'MS' - Мусиньо да Силвейра)
sex — пол студента (двоичный код: «Ж» — женский или «М» — мужской)
age - возраст ученика (числовой: от 15 до 22)
address - тип домашнего адреса студента (двоичный: 'U' - городской или 'R' - сельский)
famsize — размер семейства (двоичный: «LE3» — меньше или равно 3 или «GT3» — больше 3)
Pstatus — статус совместного проживания родителей (двоичный код: «Т» — проживание вместе или «А» — раздельно)
Medu — образование матери (числовое: 0 — нет, 1 — начальное образование (4 класс), 2 — с 5 по 9 классы, 3 — среднее образование или 4 — высшее образование)
Fedu — образование отца (числовое: 0 — нет, 1 — начальное образование (4 класс), 2 — с 5 по 9 классы, 3 — среднее образование или 4 — высшее образование)
Mjob - работа матери (номинал: «учитель», «связанный со здравоохранением», гражданские «службы» (например, административные или полицейские), «на дому» или «другое»)
Fjob - работа отца (номинал: «учитель», «связанный со здравоохранением», гражданские «службы» (например, административные или полицейские), «на дому» или «другое»)
reason - причина выбрать эту школу (номинал: близок к «дому», «репутация» школы, предпочтение «курса» или «другое»)
guardian - опекун ученика (номинальное: «мать», «отец» или «другой»)
traveltime — время в пути домой в школу (числовое: 1 – <15 минут, 2 – 15 – 30 минут, 3 – 30 минут до 1 часа или 4 – >1 часа)
Studytime — еженедельное время обучения (числовое: 1 — <2 часов, 2 — 2–5 часов, 3 — 5–10 часов или 4 — >10 часов)
failures - количество прошлых сбоев класса (числовое: n, если 1<=n<3, иначе 4)
schoolup - дополнительная образовательная поддержка (двоичный код: да или нет)
famsup — образовательная поддержка семьи (двоичный код: да или нет)
paid — дополнительные платные занятия по предмету курса (математика или португальский язык) (двоичный код: да или нет)
activities – внеклассные мероприятия (бинарный вариант: да или нет)
nursery - посещал детский сад (двоичный вариант: да или нет)
higher – хочет получить высшее образование (двоичный вариант: да или нет)
Internet — доступ в Интернет дома (двоичный код: да или нет)
romantic – с романтическими отношениями (бинарный: да или нет)
famrel - качество семейных отношений (числовое: от 1 - очень плохо до 5 - отлично)
freetime – свободное время после школы (числовое: от 1 – очень низкое до 5 – очень высокое)
goout – встреча с друзьями (числовое: от 1 – очень низкий до 5 – очень высокий)
Dalc - потребление алкоголя в течение рабочего дня (числовое: от 1 - очень низкое до 5 - очень высокое)
Walc – потребление алкоголя в выходные дни (числовое: от 1 – очень низкое до 5 – очень высокое)
health – текущее состояние здоровья (числовое: от 1 – очень плохое до 5 – очень хорошее)
absences - количество пропусков в школе (числовое: от 0 до 93)
```

Эти оценки связаны с предметом курса (математика или португальский язык):
```
G1 – оценка первого периода (числовая: от 0 до 20)
G2 – оценка за второй период (числовая: от 0 до 20)
G3 — итоговая оценка (числовая: от 0 до 20, выходная цель)
Дополнительное примечание: есть несколько (382) студентов, принадлежащих к обоим наборам данных.
Этих учащихся можно идентифицировать путем поиска идентичных атрибутов, характеризующих каждого учащегося, как показано в прилагаемом файле R.
```


Для чего датасет
====
Датасет "Student Alcohol Consumption" содержит информацию о студентах и в частности о потреблении ими алкоголя. Этот датасет может быть полезен для различных анализов и исследований в области образования и здоровья студентов. Вот несколько причин, почему этот датасет может быть полезным:

    Изучение факторов, влияющих на успеваемость студентов: Вы можете использовать этот датасет, чтобы выяснить, как различные факторы, такие как уровень образования родителей, время, проведенное за учебой, и потребление алкоголя, влияют на успеваемость студентов.

    Анализ влияния здоровья и потребления алкоголя: Датасет включает информацию о здоровье студентов и их потреблении алкоголя. Это может быть полезно для изучения связи между здоровьем и образом жизни студентов.

    Разработка стратегий для улучшения образования: Исследование этого датасета может помочь школам и учебным заведениям разработать стратегии для улучшения успеваемости студентов и их общего благосостояния.

    Предсказание успеваемости студентов: На основе данных о студентах можно попытаться разработать модель для предсказания их будущей успеваемости.

    Анализ социальных и гендерных аспектов образования: Датасет также включает информацию о социальных и гендерных характеристиках студентов, что позволяет изучать социальные тенденции в образовании.

В датасете содержатся различные числовые и категориальные признаки, которые могут быть использованы для проведения анализа и построения моделей машинного обучения. Исследование этого датасета может принести пользу как образовательным учреждениям, так и исследователям в области образования и здоровья.

Добавление новых численных показателей (колонок)
---

In [148]:
# Вычисляем среднюю оценку
df['average_grade'] = (df['G1'] + df['G2'] + df['G3']) / 3

# Вычисляем суммарное потребление алкоголя
df['total_alcohol_consumption'] = df['Dalc'] + df['Walc']

df['pass_exam'] = (df[target_column_name] > passing_grade_threshold).astype(int) # Это для бинарного

# Выводим обновленный DataFrame
print(df)

    school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0       GP   F   18       U     GT3       A     4     4   at_home   teacher   
1       GP   F   17       U     GT3       T     1     1   at_home     other   
2       GP   F   15       U     LE3       T     1     1   at_home     other   
3       GP   F   15       U     GT3       T     4     2    health  services   
4       GP   F   16       U     GT3       T     3     3     other     other   
..     ...  ..  ...     ...     ...     ...   ...   ...       ...       ...   
390     MS   M   20       U     LE3       A     2     2  services  services   
391     MS   M   17       U     LE3       T     3     1  services  services   
392     MS   M   21       R     GT3       T     1     1     other     other   
393     MS   M   18       R     LE3       T     3     2  services     other   
394     MS   M   19       U     LE3       T     1     1     other   at_home   

     ... Dalc Walc  health  absences  G1  G2  G3 av

Удаление колонок с околонулевой дисперсией
----

In [149]:
# Оставляем только числовые колонки
numerical_columns = df.select_dtypes(include=['number']).drop(columns=[target_column_name, 'pass_exam'])

# Удаляем колонки с околонулевой дисперсией
threshold = 1  # Установливаем порог
to_drop = [col for col in numerical_columns.columns if numerical_columns[col].var() < threshold]
df_cleaned = df.drop(columns=to_drop)

# Теперь df_cleaned содержит только числовые колонки без тех, у которых дисперсия меньше порога
# Выводим обновленный DataFrame
print(df_cleaned)
# Выводим убранные столбцы
print("Убрали:")
print(to_drop)

    school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0       GP   F   18       U     GT3       A     4     4   at_home   teacher   
1       GP   F   17       U     GT3       T     1     1   at_home     other   
2       GP   F   15       U     LE3       T     1     1   at_home     other   
3       GP   F   15       U     GT3       T     4     2    health  services   
4       GP   F   16       U     GT3       T     3     3     other     other   
..     ...  ..  ...     ...     ...     ...   ...   ...       ...       ...   
390     MS   M   20       U     LE3       A     2     2  services  services   
391     MS   M   17       U     LE3       T     3     1  services  services   
392     MS   M   21       R     GT3       T     1     1     other     other   
393     MS   M   18       R     LE3       T     3     2  services     other   
394     MS   M   19       U     LE3       T     1     1     other   at_home   

     ... goout Walc health absences  G1  G2  G3 ave

Удаление колонок с высокой корреляцией
---

In [150]:
# Оставляем только числовые столбцы
numeric_columns = df_cleaned.select_dtypes(include=['number']).drop(columns=[target_column_name, 'pass_exam'])
# Порог корреляции
threshold_corr = 0.9  
# Создаем корреляционную матрицу для числовых столбцов
corr_matrix = numeric_columns.corr().abs()
# Создаем булеву маску для верхнего треугольника матрицы
upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
# Ищем столбцы, которые нужно удалить
to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold_corr)]
# Удаляем столбцы с высокой корреляцией
df_cleaned.drop(to_drop, axis=1, inplace=True)
# Выводим обновленный DataFrame
print(df_cleaned)
# Выводим убранные столбцы
print("Убрали:")
print(to_drop)

    school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0       GP   F   18       U     GT3       A     4     4   at_home   teacher   
1       GP   F   17       U     GT3       T     1     1   at_home     other   
2       GP   F   15       U     LE3       T     1     1   at_home     other   
3       GP   F   15       U     GT3       T     4     2    health  services   
4       GP   F   16       U     GT3       T     3     3     other     other   
..     ...  ..  ...     ...     ...     ...   ...   ...       ...       ...   
390     MS   M   20       U     LE3       A     2     2  services  services   
391     MS   M   17       U     LE3       T     3     1  services  services   
392     MS   M   21       R     GT3       T     1     1     other     other   
393     MS   M   18       R     LE3       T     3     2  services     other   
394     MS   M   19       U     LE3       T     1     1     other   at_home   

     ... internet romantic goout Walc health absenc

Удаление выбросов
---

In [151]:
# Определение выбросов в столбце 'age' с использованием межквартильного размаха
Q1_age = df_cleaned['age'].quantile(0.25)
Q3_age = df_cleaned['age'].quantile(0.75)
IQR_age = Q3_age - Q1_age
lower_bound_age = Q1_age - 1.5 * IQR_age
upper_bound_age = Q3_age + 1.5 * IQR_age

# Определение выбросов в столбце 'absences' с использованием межквартильного размаха
Q1_absences = df_cleaned['absences'].quantile(0.25)
Q3_absences = df_cleaned['absences'].quantile(0.75)
IQR_absences = Q3_absences - Q1_absences
lower_bound_absences = Q1_absences - 1.5 * IQR_absences
upper_bound_absences = Q3_absences + 1.5 * IQR_absences

# Удаление выбросов
df_cleaned = df_cleaned[(df_cleaned['age'] >= lower_bound_age) & (df_cleaned['age'] <= upper_bound_age) &
                (df_cleaned['absences'] >= lower_bound_absences) & (df_cleaned['absences'] <= upper_bound_absences)]

# Выводим обновленный DataFrame
print(df_cleaned)

    school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0       GP   F   18       U     GT3       A     4     4   at_home   teacher   
1       GP   F   17       U     GT3       T     1     1   at_home     other   
2       GP   F   15       U     LE3       T     1     1   at_home     other   
3       GP   F   15       U     GT3       T     4     2    health  services   
4       GP   F   16       U     GT3       T     3     3     other     other   
..     ...  ..  ...     ...     ...     ...   ...   ...       ...       ...   
390     MS   M   20       U     LE3       A     2     2  services  services   
391     MS   M   17       U     LE3       T     3     1  services  services   
392     MS   M   21       R     GT3       T     1     1     other     other   
393     MS   M   18       R     LE3       T     3     2  services     other   
394     MS   M   19       U     LE3       T     1     1     other   at_home   

     ... internet romantic goout Walc health absenc

In [152]:
# Построение Box plot для столбца "absences" для измененного
fig = px.box(df_cleaned, y="absences", color_discrete_sequence = ['darkorange'], title="Распределение количества пропусков занятий")

# Настройка параметров сетки и разделения для оси y
fig.update_yaxes(
    title="Количество пропусков",
    showgrid=True,  # Отображать линии сетки для оси y
    gridcolor="lightgray",  # Цвет линий сетки
    showline=True,  # Отображать линию оси y
    linewidth=2,  # Толщина линии оси y
    mirror=True  # Отобразить линию оси y внутри графика
)

# Убераем легенду
fig.update_layout(showlegend=False)

# Отображаем Box plot
fig.show()

# Построение Box plot для столбца "absences" для изначального датафрейма
fig = px.box(df, y="absences", color_discrete_sequence = ['darkorange'], title="Распределение количества пропусков занятий")

# Настройка параметров сетки и разделения для оси y
fig.update_yaxes(
    title="Количество пропусков",
    showgrid=True,  # Отображать линии сетки для оси y
    gridcolor="lightgray",  # Цвет линий сетки
    showline=True,  # Отображать линию оси y
    linewidth=2,  # Толщина линии оси y
    mirror=True  # Отобразить линию оси y внутри графика
)

# Убераем легенду
fig.update_layout(showlegend=False)

# Отображаем Box plot
fig.show()

Идентификация
---

In [153]:
# Выбираем все столбцы с типом данных 'object' (строковые значения)
categorical_columns = df_cleaned.select_dtypes(include=['object']).columns

# Создаем объект OneHotEncoder
encoder = OneHotEncoder(drop='first', sparse=False)  # drop='first' для избегания дамми-ловушки

# Применяем кодирование к выбранным столбцам
encoded_data = encoder.fit_transform(df_cleaned[categorical_columns])

# Преобразуем результат в DataFrame
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_columns))

# Объединяем закодированные столбцы с исходным DataFrame
df_cleaned = pd.concat([df_cleaned, encoded_df], axis=1)

# Удаляем оригинальные категориальные столбцы, так как они больше не нужны
df_cleaned.drop(categorical_columns, axis=1, inplace=True)

# # Создаем бинарную целевую переменную (это мы делаем, если используем мультиклассовость)
# df_cleaned['pass_exam'] = (df_cleaned[target_column_name] >= passing_grade_threshold).astype(int)

# Разделяем данные на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(df_cleaned.drop([target_column_name, 'pass_exam'], axis=1), df_cleaned['pass_exam'], test_size=0.2, random_state=42)

# Удаляем строки с пропущенными значениями из обучающего и тестового наборов, кроме целевого столбца
X_train.dropna(subset=[col for col in X_train.columns if col != target_column_name],inplace=True)
y_train = y_train[X_train.index]  # Обновляем y_train соответствующим образом
X_test.dropna(subset=[col for col in X_train.columns if col != target_column_name],inplace=True)
y_test = y_test[X_test.index]  # Обновляем y_test соответствующим образом

# Инициализируем модель SVM с линейным ядром
model = SVC(kernel='linear')

# Обучаем модель на обучающих данных
model.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred = model.predict(X_test)

# Оценка производительности модели (например, точность)
accuracy = accuracy_score(y_test, y_pred)
print("Точность модели (%):", round(accuracy * 100, 2))

Точность модели (%): 89.86


c:\Users\ASUS\Desktop\Studie\ML\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning:

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.



In [154]:
# Выбираем все столбцы с типом данных 'object' (строковые значения)
categorical_columns = df.select_dtypes(include=['object']).columns

# Создаем объект OneHotEncoder
encoder = OneHotEncoder(drop='first', sparse=False)  # drop='first' для избегания дамми-ловушки

# Применяем кодирование к выбранным столбцам
encoded_data = encoder.fit_transform(df[categorical_columns])

# Преобразуем результат в DataFrame
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_columns))

# Объединяем закодированные столбцы с исходным DataFrame
df = pd.concat([df, encoded_df], axis=1)

# Удаляем оригинальные категориальные столбцы, так как они больше не нужны
df.drop(categorical_columns, axis=1, inplace=True)

# # Создаем бинарную целевую переменную (это мы делаем, если используем мультиклассовость)
# df['pass_exam'] = (df[target_column_name] >= passing_grade_threshold).astype(int)

# Разделяем данные на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(df.drop([target_column_name, 'pass_exam'], axis=1), df['pass_exam'], test_size=0.2, random_state=42)

# Удаляем строки с пропущенными значениями из обучающего и тестового наборов, кроме целевого столбца
X_train.dropna(subset=[col for col in X_train.columns if col != target_column_name],inplace=True)
y_train = y_train[X_train.index]  # Обновляем y_train соответствующим образом
X_test.dropna(subset=[col for col in X_train.columns if col != target_column_name],inplace=True)
y_test = y_test[X_test.index]  # Обновляем y_test соответствующим образом

# Инициализируем модель SVM с линейным ядром
model = SVC(kernel='linear')

# Обучаем модель на обучающих данных
model.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred = model.predict(X_test)

# Оценка производительности модели (например, точность)
accuracy = accuracy_score(y_test, y_pred)
print("Точность модели (%):", round(accuracy * 100, 2))

Точность модели (%): 96.2


c:\Users\ASUS\Desktop\Studie\ML\venv\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning:

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.



In [155]:
# Корреляционная матрица (оригинальный датасет)
# Удалить столбцы с текстовыми данными
dfCor = df.select_dtypes(exclude=['object'])

correlation_matrix = dfCor.corr()
fig = px.imshow(correlation_matrix, x=correlation_matrix.columns, y=correlation_matrix.index, title="Корреляционная матрица")
fig.show()

In [156]:
# Корреляционная матрица (отредактированный датасет)
# Удалить столбцы с текстовыми данными
dfCor = df_cleaned.select_dtypes(exclude=['object'])

correlation_matrix = dfCor.corr()
fig = px.imshow(correlation_matrix, x=correlation_matrix.columns, y=correlation_matrix.index, title="Корреляционная матрица")
fig.show()

1. Матрица кореляций +
2. Не бинарная, а мультиклассовая получилась (-) +
3. Создать столбец: если больше 5, то 1, а если меньше то 0, то тогда бинарная +
4. Сделать сравнение между чисткой и без (Про  идентификацию) +
5. Выгрузить на гитхаб и скинуть на почту